In [1]:
SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from ortools.linear_solver import pywraplp
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

In this nb, we solve the cutting optima problem with big order quantity to cut many specs at the same time with many stocks ( stock with same width and nearly same weight)
The condition/constraints as below: 
- `Generate Pattern` that fit in stock choice and sum each FG cut > lower bound demand
- `Filter Pattern` that keep the trim_cost <= 4%
- `Cut Pattern` that cut many stocks with filtered_pattern to minimize the stocks used

## Base Case with Duality
### 0. Helpers

In [ ]:
from typing import Dict, Any

def check_finish_weight_per_stock(weight_s: float, width_s: float, finish: Dict[str, Dict[str, Any]], BOUND_KEY: str) -> Dict[str, bool]:
    """
    DOMAIN KNOWLEDGE:
    - As upper_demand is slightly/or significant higher than need_cut, 
    - We can check to pick stock that have weight per slice cut < upper_demand
    -  So that, at least, we can cut 1 slice of the FG with that kind of stock

    PICK STOCK WITH ALL/ALMOST TRUE VALUE (SUM TRUE LARGEST)
    """
    # MC WEIGHT PER UNIT
    wu = weight_s / width_s
    # dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}
    # dict of upperbound demand FG on the chosen stock
    f_upper_demand = {f: finish[f]["upper_bound"] for f in finish.keys()}

    # dict of Yes/No 
    check_f ={f: weight_slice_f[f] < f_upper_demand[f] for f in finish.keys()}
    num_var_f = {f: 1 if weight_slice_f[f] < f_upper_demand[f] else 0 for f in finish.keys()}

    return check_f, num_var_f

In [ ]:
def make_naive_patterns(stocks, finish):
    """
    Generates patterns of feasible cuts from stock lengths to meet specified finish lengths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts = int(stocks[s]["width"] / finish[f]["width"])

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for {f}")
            return []

    return patterns


### 1. Loading data

In [4]:
## PARAMETER 
#   DOMAIN KNOWLEDGE:
# - User usually overcut current order_quantity (30 -50%) default on model
# - Users want to cut all PO qty in one stock cut ( increase boundary to max 3 months forecast)
# 
# - CASE 0
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

margin_dict = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = margin_dict[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

## PARAMETER
BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_KEY:{BOUND_KEY}, BOUND_VALUE:{BOUND_VALUE}")
LIMIT_BOUND_KEY, LIMIT_BOUND_VALUE =  "limited", None

MIN_MARGIN:6
BOUND_KEY:limited, BOUND_VALUE:None


In [5]:
# TEST DATA - NORMAL CASE , NO DEFECT
stocks = {
    "S0": {"width": 236, "weight": 1500, "qty": 1},
    "S1": {"width": 1219, "weight": 4395, "qty": 5 },
    "S2": {"width": 1219, "weight": 5260, "qty": 3},
    "S3": {"width": 1018, "weight": 3475, "qty": 3},
    "S4": {"width": 1219, "weight": 8535, "qty": 2},
    # "S5": {"width": 236, "weight": 1571, "qty": 1},
    "S5": {"width": 1219, "weight": 9260, "qty": 3},
}

finish = {
    "F1": {"width": 235, "need_cut": 800, "upper_bound": 1040},
    "F2": {"width": 147, "need_cut": 3308, "upper_bound": 3700},
    "F3": {"width": 136, "need_cut": 2290, "upper_bound": 3677},
    "F4": {"width": 68, "need_cut": 1600, "upper_bound": 1800},
    "F5": {"width": 60, "need_cut": 270, "upper_bound": 521},
    "F6": {"width": 85, "need_cut": 132, "upper_bound": 171},
    "F7": {"width": 57, "need_cut": 100, "upper_bound": 130},
    "F8": {"width": 92, "need_cut": 100, "upper_bound": 130}, 
    "F9": {"width": 57, "need_cut": 735, "upper_bound": 955},
}

### 2. Filter with list of Stocks (highest qty) can fit as-many-as possible FG

In [6]:
def filter_stock(stocks,finish):
    new_stock_list = {}
    max_f_fit = 0
    for stock_name, stock_info in stocks.items():
        _, num_var_f = check_finish_weight_per_stock(stock_info["weight"], 
                                                     stock_info["width"], finish, BOUND_KEY)
        total_f_fit = sum(num_var_f.values())

        if total_f_fit > max_f_fit:
            #new_stock_list is reset 
            new_stock_list = {stock_name: {**stock_info, "total_f_fit": total_f_fit}} 
            max_f_fit = total_f_fit
        elif total_f_fit == max_f_fit:
            new_stock_list[stock_name] = {**stock_info, "total_f_fit": total_f_fit}
    
    return new_stock_list

new_stock_list = filter_stock(stocks,finish)
new_stock_list

{'S1': {'width': 1219, 'weight': 4395, 'qty': 5, 'total_f_fit': 6},
 'S2': {'width': 1219, 'weight': 5260, 'qty': 3, 'total_f_fit': 6},
 'S3': {'width': 1018, 'weight': 3475, 'qty': 3, 'total_f_fit': 6}}

In [ ]:
# DOMAIN KNOWLEDGE:
# - FG with small spec - weight is hard to fit in a large weight stock

### Take pool of stock to optima
Assume we can take enough stock from the original list to set the demand on `cut slice` of finished goods

In [7]:
def filter_finish_demand_byslice(stock, finish: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    DOMAIN KNOWLEDGE:
    """
    # MC WEIGHT PER UNIT
    wu = stock['weight'] / stock['width']
    # Dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}

    new_finish_list = {}
    for f, finish_info in finish.items():
         new_finish_list[f] = {**finish_info
                            ,"upper_demand_slice": int(finish[f]["upper_bound"]/weight_slice_f[f])
                            , "demand_slice": int(finish[f]["need_cut"]/weight_slice_f[f])}
    
    # Filtering the dictionary to include only items with keys in keys_to_keep
    filtered_finish_list = {k: v for k, v in new_finish_list.items() if v['upper_demand_slice'] > 0}

    return filtered_finish_list

### 3. Solve with duality for pattern generation

In [8]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

# NEW PATTERN
def new_pattern_problem(finish, width_s, ap_upper_bound, demand_duals):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cost:
    #    sum{f in F} ap[f] * demand_dual[f];
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints
    # subject to stock_length:
    #    sum{f in F} ap[f] * length_f[f] <= length_s;
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s, "StockLength"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY
def generate_pattern_dual(stocks, finish, patterns):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    # c = {p: stocks[s[p]]["cost"] for p in range(len(patterns))}
    
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand_slice"] for f in F}

    # Decision variables
    # var x{P} >= 0; # relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # Objective function
    # minimize stock used:
    #         sum{p in P} x[p];
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        # sum{p in P} a[f,p]*x[p] >= demand_finish[f];
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"
        # sum{p in P} a[f,p]*x[p] <= upper_demand_finish[f];

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {
        f: max([int(stocks[s]["width"] / finish[f]["width"]) for s in stocks.keys()])
        for f in F
    }
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

In [21]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

def cut_patterns(stocks, finish, patterns):
    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(1 * x[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * x[p] for p in range(len(patterns))) >= finish[f]['demand_slice'], f"DemandSlice{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(1 * solution[p] for p in range(len(patterns)))

    return solution, total_cost

In [15]:
s = "S2"

choosen_stock_list = {k: v for k, v in new_stock_list.items() if k==s}
print(f"choosen_stock_list {choosen_stock_list}")
filtered_finish_list = filter_finish_demand_byslice(stocks['S2'],finish)
sum_needcut = sum(item['need_cut'] for item in filtered_finish_list.values())
print(f"Sum Need Cut:{sum_needcut}")

choosen_stock_list {'S2': {'width': 1219, 'weight': 5260, 'qty': 3, 'total_f_fit': 6}}
Sum Need Cut:9003


In [19]:
patterns = make_naive_patterns(new_stock_list, filtered_finish_list)

# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual(new_stock_list, filtered_finish_list, patterns)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(new_stock_list, filtered_finish_list, patterns)
    print(end=".")

print("Phase 2: Cut Pattern", end=".")

Phase 1: Pattern Generation.......Phase 2: Cut Pattern.

In [20]:
filter_trimloss_pattern = []
f_patterns = [entry for entry in patterns if entry['stock'] == s]
for pattern in patterns:
    cuts_dict= pattern['cuts']
    # print(cuts_dict)
    trim_loss = new_stock_list[s]["width"] - sum([finish[f]["width"]*cuts_dict[f] for f in filtered_finish_list.keys()])
    trim_loss_pc = trim_loss/new_stock_list[s]["width"]*100
    # weight_loss = trim_loss * new_stock_list[s]["weight"]/new_stock_list[s]["width"]
    if trim_loss_pc <= 4.0:
        print(f"***** take pattern {pattern} *****")
        filter_trimloss_pattern.append(pattern)
        print(f"trim loss percent:{trim_loss_pc}") 

***** take pattern {'stock': 'S1', 'cuts': {'F1': 5, 'F2': 0, 'F3': 0, 'F4': 0, 'F5': 0, 'F9': 0}} *****
trim loss percent:3.6095159967186223
***** take pattern {'stock': 'S2', 'cuts': {'F1': 5, 'F2': 0, 'F3': 0, 'F4': 0, 'F5': 0, 'F9': 0}} *****
trim loss percent:3.6095159967186223
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 8, 'F3': 0, 'F4': 0, 'F5': 0, 'F9': 0}} *****
trim loss percent:3.5274815422477444
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 8, 'F3': 0, 'F4': 0, 'F5': 0, 'F9': 0}} *****
trim loss percent:3.5274815422477444
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 0, 'F3': 0, 'F4': 0, 'F5': 20, 'F9': 0}} *****
trim loss percent:1.5586546349466777
***** take pattern {'stock': 'S2', 'cuts': {'F1': 0, 'F2': 0, 'F3': 0, 'F4': 0, 'F5': 20, 'F9': 0}} *****
trim loss percent:1.5586546349466777
***** take pattern {'stock': 'S1', 'cuts': {'F1': 0, 'F2': 0, 'F3': 0, 'F4': 0, 'F5': 0, 'F9': 21}} *****
trim loss percent:1.8047579983593112
***